In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

### 1. Load Data

In [8]:
with open('/Users/ChunyanHao/desktop/github/ds_take_home/data/url_list.txt') as f:
    lines = f.readlines()
    
print('Total Records:\t', len(lines))

Total Records:	 77677


In [31]:
def parse_search(lines):
    """ function to parse user's search history """
    length = len(lines)
    names = ['checkin', 'checkout', 'customMinimumPriceFilter', 
             'customMaximumPriceFilter', 'freeCancellation', 'stars_5', 
             'stars_4', 'stars_3', 'stars_2', 'stars_1', 'max_score', 
             'min_score', 'couponCode', 'adults', 'city', 'children', 
             'amenities', 'search_page']
    
    maps = {}
    for name in names:
        maps[name] = [np.nan] * length
      
    for i in range(length):
        line = lines[i]
        items = line[50:].strip().split('&')
        visited = set()
        for item in items:
            key, value = item.strip().split('=')
            key = key.strip().split('.')[1]
            if key == 'city':
                value = value.strip().replace('+', ' ')
            if key not in visited:
                maps[key][i] = value
                visited.add(key)
            else:
                maps[key][i] = maps[key][i] + ', ' + value
            
    # transform into DataFrame
    df = pd.DataFrame(maps, columns=names)
    df['checkin'] = pd.to_datetime(df['checkin'])
    df['checkout'] = pd.to_datetime(df['checkout'])
    df = df.rename(columns={'customMinimumPriceFilter': 'MinPrice', 
                            'customMaximumPriceFilter': 'MaxPrice'})
    
    return df

In [33]:
data = parse_search(lines)
data.head(3)

,checkin,checkout,MinPrice,MaxPrice,freeCancellation,stars_5,stars_4,stars_3,stars_2,stars_1,max_score,min_score,couponCode,adults,city,children,amenities,search_page
0,2015-09-19,2015-09-20,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,4,NaN,3,"New York, NY, United States",NaN,NaN,1
1,2015-09-14,2015-09-15,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,4,NaN,3,"London, United Kingdom",NaN,NaN,1
2,2015-09-26,2015-09-27,NaN,175,NaN,NaN,yes,NaN,NaN,NaN,NaN,5,NaN,2,"New York, NY, United States",NaN,NaN,1


### 2. For each search query, how many amenities were selected?